In [87]:
import numpy as np
import iicf_prediction_utils as iicfp
import iicf_similarity_matrix_utils as iicfsm

In [2]:
# Prepare MovieLens 1M Dataset

ratings = np.loadtxt('./movielens-1m/ml-1m/ratings.dat', dtype='int', delimiter='::')

movie_ids = np.array(sorted(set(ratings[:, 1])))
user_ids = np.array(sorted(set(ratings[:, 0])))

In [3]:
dat = np.zeros((len(user_ids), len(movie_ids)), dtype='int')

for ui, u in enumerate(user_ids):
    ratings_for_u = ratings[ratings[:, 0] == u]
    for ru in ratings_for_u:
        dat[ui, np.where(movie_ids == ru[1])[0][0]] = ru[2]

# Collaborative Filtering (CF) Methods

## Baseline Predictors

* do not depend on the user's ratings. Hence, **Non-personalised** baselines.
* can be used for **Pre-processing** and **Normalising** data for use with personalised algorithms.
* can be used to get over **Cold-start** problem for new users and items.

--------------------

$U$ : set of users

$I$ : set of items

$\textbf{R}$ : ratings matrix

$U_i$ : set of users that rate for item $\textit{i}$.

$I_u$ : set of items that are rated by user $\textit{u}$. 

$r_{u,i}$ : rating of user $\textit{u}$ for item $\textit{i}$.

$b_{u,i}$ : baseline prediction for user $\textit{u}$ and item $\textit{i}$.

### Simplest Baseline Method


$b_{u,i} = \mu = \frac{1}{|U|} \sum_{u} \frac{1}{|I_u|} \sum_{i \in I_u} r_{u,i}$ : average rating over all ratings in the system.

$b_{u,i} = \hat{r_u} = \frac{1}{|I_u|} \sum_{i \in I_u} r_{u,i}$ : average rating for user $\textit{u}$.

$b_{u,i} = \hat{r_i} = \frac{1}{|U_i|} \sum_{u \in U_i} r_{u,i}$ : average rating for item $\textit{i}$.

### General Baseline Method

* combines the **user average rating** ($\hat{r_u} = \mu + b_u$) with the **average deviation** ($b_i$) from user mean rating for a particular item [14, 58, 79, 110, 115].

-----------------

$b_u$ : user baseline predictor.

$b_i$ : item baseline predictor.

-----------------

$b_{u,i} = \mu + b_u + b_i$

$b_u = \frac{1}{|I_u|} \sum_{i \in I_u} (r_{u,i} - \mu) = \hat{r_u} - \mu$

$b_i = \frac{1}{|U_i|} \sum_{u \in U_i} (r_{u,i} - b_u - \mu) = \hat{r_i} - \mu - \frac{1}{|U_i|} \sum_{u \in U_i} b_u = \hat{r_i} - \frac{1}{|U_i|} \sum_{u \in U_i} \hat{r_u}$

Hence

$b_{u,i} = \hat{r_i} + \hat{r_u} - \frac{1}{|U_i|} \sum_{u' \in U_i} \hat{r_{u'}}$

------------------

* can be further regularised, providing a more reasonable estimate of user and item preferences in the face of sparse sampling, with the incorporation of **damping terms** $\beta_u$ and $\beta_i$ [44]. this adjustment causes the baseline predicted ratings to be closer to global mean when the user or item has few ratings. Funk [44] found that **25** was a useful value for the damping terms.

------------------

$b_u = \frac{1}{|I_u| + \beta_u} \sum_{i \in I_u} (r_{u,i} - \mu)$

$b_i = \frac{1}{|U_i| + \beta_i} \sum_{u \in U_i} (r_{u,i} - b_u - \mu)$

$\beta_u = \beta_i = 25$

-------------

* additional baselines can be computed and added, and the baselines can be made more sophisticated to deal with various effects [14, 80, 115].

* if an item or user has no ratings, its baseline can be set to **0**, effectively assuming that it is an average user or item.

* baseline predictors effectively capture effects of **user bias**, **item popularity**, and can be applied to more exotic but increasingly-important factors such as time [80, 115].

* if the baseline is subtracted from the $\textbf{R}$ to yield a **normalised ratings matrix** $\hat{\textbf{R}}$, all that remains for CF to do is to efficiently capture the interaction effect between users and items. Further, the missing values of $\hat{\textbf{R}}$ are **0** rather than unknown, simplifying some computations and allowing the matrix to be handled by standard sparse matrix packages.

### Obtaining Normalized Ratings Matrix $\hat{\textbf{R}}$ for MovieLens 1M Dataset

* missing values of data should be assigned to 0.


* if no rating exists for a user or an item, corresponding baseline ($b_u, b_i$) should be assigned to 0. This is not a case in this dataset.


* this is preprocessing step to calculate similarity matrix for **real-valued** rating domain. If rating domain is **unary**, to implement appropriate preprocessing algorithm, look at user-user CF or item-item CF sections. 

#### Exercise:

1. calculate baselines with damping terms $\beta_u = \beta_i = 25$ and obtain normalized ratings matrix.
2. calculate baselines with damping terms and obtain normalized ratings matrix after adding a new user and a new item without rating values to dataset.

In [4]:
mu = np.mean([d for d in dat.flatten() if d != 0])
b_u = np.array([np.mean([u_i_r - mu for u_i_r in u if u_i_r != 0]) for u in dat])
b_i = np.array([np.mean([i_u_r - mu - b_u[u_id] for u_id, i_u_r in enumerate(i) if i_u_r != 0]) for i in dat.T])

In [5]:
baseline_model = np.zeros(dat.shape, dtype='float')

for u in xrange(baseline_model.shape[0]):
    for i in xrange(baseline_model.shape[1]):
        if dat[u, i] == 0:
            baseline_model[u, i] = 0
        else:
            baseline_model[u, i] = mu + b_u[u] + b_i[i]

In [6]:
# To predict a missing value with baseline model calculate "mu + b_u[user_index] + b_i[item_index]"

print '** Prediction for User ID {} and Movie ID {}'.format(user_ids[0], movie_ids[1])
print 'Original Rating : {}'.format(dat[0][1])
print 'Predicted Rating : {}'.format(mu + b_u[0] + b_i[1])
print '\n** Prediction for User ID {} and Movie ID {}'.format(user_ids[5], movie_ids[1])
print 'Original Rating : {}'.format(dat[5][1])
print 'Predicted Rating : {}'.format(mu + b_u[5] + b_i[1])

** Prediction for User ID 1 and Movie ID 2
Original Rating : 0
Predicted Rating : 3.86453658693

** Prediction for User ID 6 and Movie ID 2
Original Rating : 0
Predicted Rating : 3.57726579235


In [7]:
normalized_ratings = dat.astype('float') - baseline_model

## User-User Collaborative Filtering (UUCF)

## Item-Item Collaborative Filtering (IICF)

* UUCF, while effective, suffers from **scalability** problems as the user base grows. searching for the neighbours of a user is an $O(|U|)$ operation (or worse, depending on how similarities are computing $-$ directly computing most similarity functions against all other users is linear in the total number of ratings). IICF [71, 87, 130] takes a major step in this direction and is one of the most widely deployed CF techniques today.

------------------

* UUCF : uses similarities between users' rating behaviour to predict preferences.


* IICF : uses similarities between the rating patterns of items to predict preferences. if two items tend to have the same users like and dislike them, then they are similar and users are expected to have similar preferanes for similar items.

------------------

* similar to **Content-Based Filtering** for recommendation and presonalization, but item similarity is deduced from user preference patterns rather than extracted from item data.

* necessary to find the most similar items (solving k-NN problem) to generate predictions and recommendations. If $|U| >> |I|$, then it allows the neighbourhood-finding to be amongst the smaller of the two dimensions, but this is a **small gain**.

* provides major performance gains by lending itself well to **pre-computing the similarity matrix**.

---------------------

* UUCF : as a user rates and re-rates items, their rating vector will change along with their similarity to other users. finding similar users in advance is therefore complicated: a user's neighbourhood is determined not only by their ratings but also by the ratings of other users, so their neighbourhood can change as a result of new ratings supplied by any user in the system. for this reason, most UUCF systems **find neighbourhoods at the time** when predictions or recommendations are needed.


* IICF : in systems with a sufficiently high $\frac{|U|}{|I|}$, one user adding or changing ratings is unlikely to significantly change the similarity between two items, particularly when the items have many ratings. therefore, it is reasonable to **pre-compute similarities between items** in an item-item similarity matrix. the rows of this matrix can even be **truncated to only store the** $k$ **most similar items**. As users change ratings, this data will become **slightly stale**, but the users will likely still receive good recommendations and the data can be **fully updated by re-computing the similarities** during a **low-load time** for the system.

-----------------------

* UUCF : generates predictions by using **other users' ratings for the target item** combined with **user similarities**.


* IICF : generates predictions by using the **user's own ratings for other items** combined with **those items' similarities to the target item**.

-------------------------

* system needs;

    1. a similarity function $s : I$ x $I$ 
    2. a method to generate predictions from ratings and similarities.
    
### Computing Predictions 

* in **real-valued ratings domains**, the similarity scores can be used to generate predictions using a **weighted average**.

* recommendations are generated by picking the candidate items with the highest predictions.

-----------------

$S$ : set of items similar to item $i$. 

$p_{u,i}$ : predicted value

----------------------

$p_{u,i} = \frac{\sum_{j \in S} s(i,j) r_{u,j}}{\sum_{j \in S} |s(i,j)|}$

* $S$ is typically selected to be the $k$ items most similar to $i$ that $u$ has also rated for some neighbourhood size $k$. Sarwar et al. [130] found $k = 30$ produced good results on the **MovieLens** dataset.


* this equation suffers from two deficiencies;
    1. when it is possible for similarity scores to be **negative** and ratings are constrained to be **non-negative**, some of the ratings averaged to compute the prediction may be **negative** after weightings. while this will **not affect the relative ordering of items** by predicted value, it will **bias the predicted values so they no longer map back to the user rating domain**. this can be corrected either by **thresholding** similarities so only items with **non-negative** similarities are considered or by **averaging** distance from the baseline predictor. in latter case, above equation becomes;
    
    $p_{u,i} = \frac{\sum_{j \in S} s(i,j) (r_{u,j} - b_{u,i})}{\sum_{j \in S} |s(i,j)|} + b_{u,i}$
    
    2. when rating scales are **non-real-valued** - particularly **unary** scales are common on e-commerce sites without ratings data - the averaging does not work. if all similarities are positive and $r_{u,i} = 1$ if user $u$ has purchased item $i$, then base equation to compute predictions always evaluates to **1**. Also, with negative similarities, it is similarly **ill-behaved**. to work around this, we can compute **pseudo-predictions** $\tilde{p}_{u,i}$ with a simple aggregation of the similarities to items in the user's purchase history $I_u$. **Summation** has been tested and shown to perform well; other aggregations such as **mean** or **max** may also be considered or used in practice.
    
    $\tilde{p}_{u,i} = \sum_{j \in I_u} s(i, j)$
    
    $\tilde{p}_{u,i}$ is not in a meaningful scale to predict any particular user behaviour, but the predict task is typically not as important in unary contexts. This preudo-prediction can, however, be used to **rank** candidate items for recommendation, forming a good basis for using IICF to recommend items based on user purchase histories [38, 71, 87].

### Computing Item Similarity

* $\textbf{S}$ : item-item similarity matrix. $\textbf{S}$ is a standard **sparse matrix**, with missing values being **0** (no similarity); it differs in this respect from $\textbf{R}$, where missing values are unknown.

#### Cosine similarity

* cosine similarity between item rating vectors is the most popular similarity metric, as it is **simple**, **fast**, and produces good predictive accuracy;

$s(i,j) = \frac{\textbf{r}_i \cdot \textbf{r}_i}{||\textbf{r}_i||_2 ||\textbf{r}_j||_2}$

#### Conditional Probability [TODO]

* for domains with **unary** ratings (such as shopping site purchase histories), Karypis [71] proposed a similarity function based on conditional probabilities: 

#### Pearson Correlation

* pearson correlation has also been proposed for item-item recommendation, but does not seem to work as well as cosine similarity [130].

--------------------------------

* preprocessing:

    * in order to optimize the recommender's performance, it is important to **normalise** ratings** $\textbf{R}$ **prior to computing the similarity matrix [14, 130]. 
    
    * in **real-valued** domains variation in ratings due to **user ratings bias** (e.g., two users liking and disliking similar films, but one being a cynic who rates average films 2.5/5 and the other an enthusiast who rates the average film at 4/5) and **item bias** allows the CF to focus on the more nuanced differences in user preference for particular items. this can be accomplished by **subtracting a baseline predictor from all ratings prior to computing similarities** (e.g., compute similarities over ratings $\hat{r}_{u,i} = r_{u,i} - \mu - b_u - b_i$).

    * when applying **cosine similarity** in **unary** ratings domains, it can be useful to **normalise each user's ratings vector** $r_u$ **to the unit vector prior to computing item similarities**. the effect of this adjustment is that users who have purchased fewer items have more impact on the similarity of the items they have purchased than users who have purchased many items [38, 71].
    
    
* postprocessing:

    * **normalising item similarity vectors** (rows of $\textbf{S}$) **to unit vectors** can be beneficial. this causes items with **sparser neighbourhoods** (fewer similar items) to have more influence in computing the final predictions [38, 71].
    * [TODO] Question: normalisation does not affect predictions as can be seen in prediction equations.

### Pre-computing and Truncating the Model

* due to the relatively static nature of item similarities when $|U| >> |I|$, it is feasible to **pre-compute** item-item similarities and **cache the** $k'$ **most similar items to each item**. prediction can then be performed quickly by looking up the similarity list for each item rated by the current user and aggregating their similarities into a predicted preference.

* caching more items than are used in the similarity computation (so $k' > k$) is useful to increase the likelihood of having $k$ similar items after items already rated by the user have been removed from the candidate set.

* **pre-computation** and **truncation** is essential to depoloying collaborative filtering in practice, as it **places an upper bound on the number of items which must be considered to produce a recommendation** and **eliminates the query-time cost of similarity computation**. itcomes with the small expense of **reducing the number of items for which predictions can be generated** (the coverage of the recommender), but the unrecommendable items will usually have low predicted preferences anyway. 

|Process                         |**Real-Valued** Rating Domain                  |**Unary** Rating Domain|
|:------------------------------:|:---------------------------------------------:|:-----:|
| Normalisation of $\textbf{R}$  | using **baseline predictor** in first section |  **normalise** each user's ratings vector $r_u$ to the **unit vector** (if **cosine similarity** or **pearson correlation** is used)|
| Postprocessing of $\textbf{S}$ | **normalising** item similarity vectors (rows of $\textbf{S}$) to **unit vectors**    | **normalising** item similarity vectors (rows of $\textbf{S}$) to **unit vectors**|
| Similarity Metrics             | **cosine similarity** or **pearson correlation** | **cosine similarity**, **pearson correlation** or **conditional probability**|
| Computing Predictions | $p_{u,i} = \frac{\sum_{j \in S} s(i,j) (r_{u,j} - b_{u,i})}{\sum_{j \in S} |s(i,j)|} + b_{u,i}$ (or **thresholding** instead of **averaging distance** from base predictor)| $\tilde{p}_{u,i} = \sum_{j \in I_u} s(i, j)$ (**mean** or **max** can be used instead of **summation**)|
| Selection of $k$ | -- | -- |
| Selection of $k'$ when **truncating** | -- | -- |

### Obtaining Similarity Matrix $\textbf{S}$ for MovieLens 1M Dataset and Recommendation

* missing values of similarity matrix will be 0, which represents no similarity.

#### Exercise:

1. use pearson correlation instead of cosine similarity as similarity metric.
2. truncate similarity matrix to appropriate $k'$ items.

#### Get Similarity Matrix $\textbf{S}$ and Normalise

In [10]:
similarity_matrix = iicfsm.get_similarity_matrix(normalized_ratings)
similarity_matrix = iicfsm.normalize_similarity_matrix(similarity_matrix)

#### Predict Rating of a User for an Item

* Sarwar et al. [130] found k=30 produced good results on the MovieLens dataset.

In [112]:
def predict(predict_user_index, predict_item_index):
    
    k_most_similar = 30
    
    user_id = user_ids[predict_user_index]
    movie_id = movie_ids[predict_item_index]

    original_rating = dat[predict_user_index][predict_item_index]

    avg_dist_prediction = iicfp.predict_with_avg_distance(predict_user_index, predict_item_index, dat, 
                                                          similarity_matrix, k_most_similar, baseline_model)

    thresholded_prediction = iicfp.predict_with_thresholding(predict_user_index, predict_item_index, dat, 
                                                             similarity_matrix, k_most_similar)

    print '\n** Prediction for User ID / Movie ID                            : {} / {}'.format(user_id, movie_id)
    print '** Original Rating                                              : {}'.format(original_rating)
    print '-- Prediction by Averaging Distance from the Baseline Predictor : {}'.format(avg_dist_prediction)
    print '-- Prediction by Thresholding Similarities                      : {}'.format(thresholded_prediction)

#### Get Most (Dis-)Simillar Items to defined Item Using Similarity Matrix and Predict Ratings of those Items for defined User

In [113]:
def sim_n_dissim(predict_item_index):
    
    movie_id = movie_ids[predict_item_index]

    most_similar_item, similarity_value = iicfp.get_similar_items_using_similarity_matrix(predict_item_index, 
                                                                                          similarity_matrix, 1)
    most_similar_item = most_similar_item[0]
    similarity_value = similarity_value[0]
    most_similar_movie_id = movie_ids[most_similar_item]

    most_dissimilar_item, dis_similarity_value = iicfp.get_dissimilar_items_using_similarity_matrix(predict_item_index, 
                                                                                                    similarity_matrix, 1)
    most_dissimilar_item = most_dissimilar_item[0]
    dis_similarity_value = dis_similarity_value[0]
    most_dissimilar_movie_id = movie_ids[most_dissimilar_item]

    print '\n** Prediction for Movie ID {}'.format(movie_id)
    print '-- Most Similar Movie Index    : {:6d}, Movie ID : {:6d}, Similarity Value : {}'.format(most_similar_item,
                                                                                             most_similar_movie_id,
                                                                                             similarity_value)
    print '-- Most Dissimilar Movie Index : {:6d}, Movie ID : {:6d}, Similarity Value : {}'.format(most_dissimilar_item,
                                                                                             most_dissimilar_movie_id,
                                                                                             dis_similarity_value)

In [139]:
predict(321, 259)
sim_n_dissim(259)


** Prediction for User ID / Movie ID                            : 322 / 266
** Original Rating                                              : 5
-- Prediction by Averaging Distance from the Baseline Predictor : 4.56400745677
-- Prediction by Thresholding Similarities                      : 4.56400745677

** Prediction for Movie ID 266
-- Most Similar Movie Index    :   2668, Movie ID :   2875, Similarity Value : 0.064064557291
-- Most Dissimilar Movie Index :   1060, Movie ID :   1138, Similarity Value : -0.0464289192239


In [140]:
predict(321, 2668)
predict(321, 1060)


** Prediction for User ID / Movie ID                            : 322 / 2875
** Original Rating                                              : 0
-- Prediction by Averaging Distance from the Baseline Predictor : 4.47743707569
-- Prediction by Thresholding Similarities                      : 4.55924405334

** Prediction for User ID / Movie ID                            : 322 / 1138
** Original Rating                                              : 0
-- Prediction by Averaging Distance from the Baseline Predictor : -1.93244620498
-- Prediction by Thresholding Similarities                      : 4.60015785374


## Dimensionality Reduction

* an item is a $|U|$-dimensional vector with missing values of users' preferences for it (similarly, a user is a $|I|$-dimensional vector). there is redundancy in these dimensions, as both users and items will usually be divisible into groups with similar preference profiles. it is therefore natural to ask whether the dimensionality of the rating space can be reduced - **can we find a smaller number of dimensions, ideally a constant number $k$, so that items and users can be represented by $k$-dimensional vectors**?

### Defining Singular Value Decomposition (SVD)

* for a matrix $\textbf{M}$, its SVD is the factorisation of $\textbf{M}$ into three constituent matrices such that $\textbf{M} = \textbf{U}\Sigma\textbf{T}^T$. $\Sigma$ is a **diagonal matrix** whose values $\sigma_i$ are the **singular values** of the decomposition, and both $\textbf{U}$ and $\textbf{T}$ are **orthogonal**. this decomposition introduces an intermediate vector space represented by $\Sigma$.


* if $\textbf{M}$ is the **ratings matrix**, $\Sigma\textbf{T}^T$ transforms vectors from **item-space** into the **intermediate vector space**.


* in the pure form of the SVD
    * $\textbf{M}$ : [m x n] and has rank $\hat{k}$
    * $\textbf{U}$ : [m x $k$]
    * $\Sigma$     : [k x $\hat{k}$]
    * $\textbf{T}$ : [n x $\hat{k}$]


* $\Sigma$ can be **truncated** by only retaining the $k$ largest singular values to yield $\Sigma_k$. resulting decomposition is an approximation of $\textbf{M}$. further, using the **Frobenius norm as the measure of error**, it is the best possible **rank-$k$ approximation** [36].


* this truncation simultaneously achieves two goals;
    1. it **decreases the dimensionality** of the vector space, **decreasing the storage** and **computational requirements** for the model. items and users can each be represented by $k$-dimensional vectors.
    2. by dropping the **smaller singular values**, small preturbances as a result of **noise in the data are eliminated**, leaving only the **strongest effects** or **trends** in the model.
    

* computing the SVD of the ratings matrix results in the following factorisation ($m = |U|$, $n = |I|$, and $\sigma$ : [$k$ x $k$] diagonal matrix)

$\textbf{R} \approx \textbf{U}\Sigma_k\textbf{T}^T$


* the rows of the $|U|$ x $k$ matrix $\textbf{U}$ are the users' interest in each of the $k$ **inferred topics (inferred items)**, and the rows of $\textbf{T}$ are the item's relevance for each **inferred topic**. the singular values in $\Sigma$ are **weights for the preferences**, representing the influence of a particular **inferred topic** on user-item preferences across the system.


* a user's preference for an item, therefore, is the **weighted sum of the user's interest in each of the inferred topics times that item's relevance to the inferred topic**.

### Computing and Updating the SVD

* in order to use SVD (or any matrix factorisation method), it is necessary to first compute the matrix factorisation. there are a variety of algorithms for computing SVD, including **Lanczos' algorithm**, **the generalized Hebian algorithm**, and **expectation maximisation** [51, 81, 127].


* SVD is only **well-defined** when the matrix is **complete**. Therefore, to factor the rating matrix, the missing values must be filled with some reasonable default (a method called **imputation**). Sarwar et al. [131] found the **item's average rating** to be a useful default value (they tried user average as well, but item average performed better). Alternatively, the SVD can be computed over the **normalised ratings matrix** $\hat{\textbf{R}}$ and the missing values considered to be **0**.


* several methods have been proposed that compute an estimate of the **SVD only on the known ratings**, dropping the requirement to impute or otherwise account for missing ratings. Kurucz et al. [81] propose a **least-squares method** that learns a regression for each user. another method that has become quite **popular in the last few years** is **gradient descent** [44, 110]. this method **trains** each topic $f$ in turn, using the following **update rules**;

$\lambda = 0.001$ (learning rate)

$\Delta u_{j,f} = \lambda (r_{u,i} - p_{u,i}) i_{k,f}$

$\Delta i_{k,f} = \lambda (r_{u,i} - p_{u,i}) u_{j,f}$


* the gradient descent method for estimating the SVD also allows for **regularisation** to prevent **overfitting** the resulting model. The resulting model will not be a **true** SVD of the rating matrix, as the component matrices are **no longer orthogonal**, but tends to be **more accurate** at predicting unseen preferences than the unregularised SVD. the regularisation is accomplished by adding an additional term to the update rules. $\gamma$ is the **regularisation factor**, typically $0.1$-$0.2$.

$\Delta u_{j,f} = \lambda ((r_{u,i} - p_{u,i}) i_{k,f} - \gamma u_{j,f})$

$\Delta i_{k,f} = \lambda ((r_{u,i} - p_{u,i}) u_{j,f} - \gamma i_{k,f})$

* prior to computing the SVD, the ratings can additionally be **normalised**, by subtracting the **user's mean rating** or **some other baseline predictor**. this can **improve both accuracy** [131] and **accelerate convergence of iterative methods**.


* once the SVD is computed, it is necessary to **update** it to reflect **new users**, **items**, and **ratings**. a commonly used method for updating the SVD is **folding-in**; it works well in practive and **allows users who were not considered when the ratings matrix was factored to receiver recommendations and predictions** [16, 129]. **Folding-in** operates by computing a **new user-preference** or **topc-relevance** vector for the new user or item but not recomputing the decomposition itself.


* for a user $u$, **folding-in** computes their topic interest vector $u$ such that


### Generating Predictions 

### Computing Similarities

### Normalisation in SVD

### Principle Component Analysis (PCA)